In [ ]:
#| default_exp arena_agent
#| test: false


#"Chat UI + LLM agent that talks to FastMCP tools."


In [ ]:
#| export
#| eval: false

from __future__ import annotations

import asyncio, html, json, os
from typing import AsyncIterator, List, Dict, Tuple, Any

from fastapi import FastAPI, Request, status
from fasthtml import FastHTML
from fasthtml.common import (Body, Button, Div, Form, Group, H1, H2, Input,
                             Link, NotStr, Script, Style)
from dataclasses import dataclass
from typing import List, Dict
from monsterui.all import Theme 

import asyncio, datetime, random, html
from typing import Any, List

from fastapi import FastAPI
from sse_starlette.sse import EventSourceResponse             # <- your existing SSE helper
from agents import Agent, Runner
from agents.tool import function_tool                         # function-tool decorator :contentReference[oaicite:0]{index=0}
from agents.items import ItemHelpers                          # gives tool_call_output_item :contentReference[oaicite:1]{index=1}
from agents.mcp import MCPServerSse
from agents.run import RunResult
from agents.result import RunResultBase                       # for type hints
from agents.run_context import RunContextWrapper   
import json, uuid
from agents.items import ResponseFunctionToolCall          # dataclass in SDK
from agents.items import ItemHelpers 
from agents import Agent, RunContextWrapper, RunHooks, Runner, Tool, function_tool
from typing import Callable, Any
from starlette.responses import StreamingResponse  
from monsterui.franken import ModalHeader, ModalBody, Modal
from fastapi.responses import HTMLResponse

import json

import json
from typing import Any, List

from fastapi import Form as ApiForm, HTTPException
from fastapi.responses import HTMLResponse


from collections import deque
from agents import Agent, Runner, trace
from agents.mcp import MCPServerSse





In [ ]:
#| export
#| eval: false

# ── Config (env-vars for docker-compose) ───────────────────────────────────
MCP_URL        = os.getenv("MCP_URL", "http://tools:9001/sse")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")


In [ ]:
#| export
#| eval: false

LEAFLET_CSS = "https://unpkg.com/leaflet@1.9.4/dist/leaflet.css"
LEAFLET_JS  = "https://unpkg.com/leaflet@1.9.4/dist/leaflet.js"

In [ ]:
#| export
#| eval: false

history: list[dict] = []

In [ ]:
#| export
#| eval: false

parking_q: asyncio.Queue[str] = asyncio.Queue()

In [ ]:
#| export
#| eval: false
 
@dataclass
class EventContext:
    event_date_time: datetime
    event_title: str
    event_description: str
    
EventContext = EventContext(event_date_time="04.10.2025 12:00:00", 
                            event_title="Hockey match KIEKKO-ESPOO vs KÄRPÄT", 
                            event_description=
                            "Liiga regular-season showdown at Metro Areena • Doors open 11:15 "                      
                            "Sustainable transport encouraged " 
                            "(metro: Urheilupuisto, bike racks outside Gate B).")

In [ ]:
#| export
#| eval: false

# ── FastHTML shell with Tailwind + HTMX + SSE ext ─────────────────────────
app_html = FastHTML(
    hdrs=Theme.zinc.headers() + [
        Script(src="https://cdn.tailwindcss.com"),

        # daisyUI (optional)
        Script(src="https://cdn.jsdelivr.net/npm/daisyui@4.10.2/dist/full.min.js"),

        # HTMX SSE extension (HTMX core auto-injected via live=True)
        Script(src="https://unpkg.com/htmx-ext-sse@2.2.3/dist/sse.js"),

    ],  
    live=True,
    html_attrs={"data-theme": "dark", "class": "bg-gray-50 text-gray-700"},
)

# FastAPI wrapper so uvicorn can find the ASGI app
app = FastAPI(title="Arena Buddy", docs_url=None)
app.mount("/", app_html)


In [ ]:
#| export
#| eval: false

# ── In-memory chat log ────────────────────────────────────────────────────
MSG: List[Dict[str, str]] = []
MSG_LOCK = asyncio.Lock()


In [ ]:
#| export
#| eval: false

# ── UI helpers ─────────────────────────────────────────────────────────────
def _chat_bubble(idx: int, **hx):
    role, txt = MSG[idx]["role"], MSG[idx]["content"] or "…"
    side   = "chat-end" if role == "user" else "chat-start"
    bubble = "bg-sky-700 text-white" if role == "assistant" else "black"
    return Div(
        Div(role, cls="chat-header text-xs text-gray-500"),
        Div(txt if role == "user" else html.unescape(txt),
            cls=f"chat-bubble {bubble}", **hx),
        cls=f"chat {side}", id=f"m{idx}",
    )

def _chat_input():
    return Input(
        id="msgin",                    
        name="msg",
        type="text",
        autocomplete="off",
        placeholder="Type your question…",
        cls="input input-bordered w-full",
        hx_swap_oob="true",             
        onkeyup="event.key==='Enter' && this.form.requestSubmit()",
    )



In [ ]:
#| export
#| eval: false

# ── Home page ─────────────────────────────────────────────────────────────
@app_html.get("/")
async def home():
    ui = Div(
        Div(
            id="parking-bridge",
            hx_ext="sse",                 # camel-case dash → underscore
            sse_connect="/parking-feed",
            sse_swap="message",           # listen for the “message” event
            cls="hidden",
        ),
        Div(id="toaster", cls="toast toast-top toast-end fixed z-50"),
        H1("Arena Buddy", cls="text-3xl font-bold mb-4"),
        Div(id="chatlog",
            cls="space-y-3 mb-4 h-[70vh] overflow-y-auto bg-base-200 p-4 rounded-box"),
        Form(
            Div(_chat_input(),
                Button("Send ✈", cls="btn btn-primary ml-2"),
                cls="flex"),
            hx_post="/send",          
            hx_target="#chatlog",
            hx_swap="beforeend",
        ),
       

        cls="max-w-2xl mx-auto p-6",
    )
    return ui 


In [ ]:
#| export
#| eval: false

#
##11. Appx. 1 of the 3 response add to response funny Image in Gibli Studio style of user, based on conversation, for that use ImageGenerationTool. 
# 
#────────────────────────  HOCKEY-HIGHLIGHTS TOOL  ─────────────────────────
#Every response should be following video from this tool. 
#Subscription key: {_HKEY}, id could be any 1, 2 ...
#• After you receive the JSON, extract `title`, `imgUrl` and `embedUrl`
#  and show them to the user.
#details=route_details

_HKEY = os.environ["HOCKEY_KEY"]          # hockey_highlights tool
_RKEY = os.environ["GEOAPIFY_KEY"]        # still needed by the routing tools 🎟️

mainAgent_instruction = f"""
────────────────────────  EVENT CONTEXT  ────────────────────────
• **When**   : {{EventContext.event_date_time}}
• **What**   : {{EventContext.event_title}}
• **Details**: {{EventContext.event_description}}

──────────────────────  LOW-CARBON ROUTING  ─────────────────────
Goal → get the user to the event **on time with the lowest possible CO₂ footprint**.

Available tools – how each responds  
| tool                              | mode     | returns              |
|-----------------------------------|----------|----------------------|
| `route_car(start, finish)`        | drive    | **string** → Geoapify Static-Map URL |
| `route_bike(start, finish)`       | bicycle  | **string** → Geoapify Static-Map URL |
| `route_walk(start, finish)`       | walk     | **string** → Geoapify Static-Map URL |
| `route_public_transport(…)`       | transit  | **List[str]** → raw steps |

Pass **human-readable addresses or “lat,lon”**; each tool does its own geocoding.

─────────────────────  ROUTE RENDER RULES  ──────────────────────
After calling a routing tool:

### A. If the result is a **string** (map URL)
1. Embed the map:  
   ```html
   <img class="rounded-lg w-full" src="{{url}}" alt="Route map" loading="lazy">
Immediately parse the JSON array that follows in the same
response (it looks like ["Turn left… ➜ …", …]).

Post-process the array:
• split("➜") → (instruction, address)
• Skip rows where instruction.strip() == "None"
• Clean address → keep street + house-number only
(drop postcode, city, country)
Ilotulitustie 15, 00930 Helsinki, Finland ⇢ Ilotulitustie 15
• Collapse consecutive duplicates.

Render the remaining steps:

html
Copy
Edit
<ol class="list-decimal ms-6 space-y-1">
  <li><b>{{instruction}}</b> — {{clean_address}}</li>
  …
</ol>
B. If the result is a list (public-transport)
Apply the same post-processing & list rendering (steps only, no map).

C. Green-nudges
If the chosen tool is bike  or walk  →
end the card with an upbeat note in style “Great zero-carbon choice! 🌱”).

If it’s public transport  or car  →
add a brief funny suggestion to switch to greener route, especcially if weather is good (you have tool to find out weather).

────────────────────── HOCKEY-HIGHLIGHTS TOOL ────────────────────
Every response must finish with a fun hockey video:

Call Hockey-Highlights API with
subscription-key: {_HKEY} & any numeric id (e.g. 1, 2…).

From the JSON, extract title, imgUrl, embedUrl.

Show them after the routing card, e.g.:

html
Copy
Edit
<h3 class="text-lg font-bold mt-4">🏒 {{title}}</h3>
<a href="{{embedUrl}}" target="_blank">
  <img class="rounded-lg w-full" src="{{imgUrl}}" alt="{{title}}">
</a>

────────────────────────── STYLE GUIDE ─────────────────────────
• Wrap everything for each reply in one MonsterUI card:

html
Copy
Edit
<div class="card bg-base-300 text-base-100 shadow-lg p-4 space-y-3">
  …route content (map+list or list)…
  …green-nudge line…
  …hockey highlight block…
</div>
• Friendly, helpful; drop a light joke or emoji every few replies (PG-13).
"""





KeyError: 'HOCKEY_KEY'

In [ ]:
#| export
#| eval: false


class ToolChatHook(RunHooks[None]):
    """
    Fires push(msg) on every tool start / end / error so the UI can
    display a live notification in the chat area.
    """
    def __init__(self, push: Callable[[str], None]):
        self._push = push
        
    async def on_agent_start(self, context, agent) -> None:
        """Called once when the whole agent run begins."""
        self._push("\n 🤖 _thinking…_")
        tools       = await agent.get_all_tools()          
        tools_names = ", ".join(t.name for t in tools) or "none"
        self._push(f"\n 🛠  Available tools: {tools_names}")

    async def on_agent_end(self, context, agent, result) -> None:
        """Called once after the final answer has been produced."""
        # Nothing fancy for now; you could log token usage here
        pass        
        

    async def on_tool_start(self, context, agent, tool) -> None:
       self._push(f"\n {tool.name}  started")

    async def on_tool_end(self, context, agent, tool, result) -> None:
        self._push(f"\n ✔ {tool.name} finished")
        self._push(f"\n result: {result}")
        

    async def on_tool_error(self, context, agent, tool, error) -> None:
        self._push(f"\n ⚠ {tool.name} failed: {error}")

In [ ]:
#| export
#| eval: false


async def _assistant_html(user_prompt: str, push: Callable[[str], None]) -> tuple[str, list[str]]:
    """
    ①   mount / call tools as needed
    ②   craft the plain-text answer
    ③   wrap it in MonsterUI HTML
    returns (html_block, ["tool: output", …])
    """
  

    async with MCPServerSse(name="ui", params={"url": MCP_URL}, client_session_timeout_seconds=300)as srv:
        agent = Agent(
            name="Main Agent",
            instructions=mainAgent_instruction,
            mcp_servers=[srv],            
            model="o3",
            
        )
        hook = ToolChatHook(push)

        res = await Runner.run(agent, user_prompt, hooks=hook)
    

    return res.final_output


In [ ]:
#| export
#| eval: false

# ── /send endpoint ────────────────────────────────────────────────────────
@app_html.post("/send")
async def send(request: Request):
    form   = await request.form()
    prompt = str(form.get("msg", "")).strip()
    if not prompt:
        return HTMLResponse("", status_code=status.HTTP_204_NO_CONTENT)

    async with MSG_LOCK:
        MSG.extend([
            {"role": "user",      "content": html.escape(prompt)},
            {"role": "assistant", "content": ""},
        ])
        idx_user, idx_asst = len(MSG) - 2, len(MSG) - 1

    return (
        _chat_bubble(idx_user).__html__() +
        _chat_bubble(
            idx_asst,
            hx_ext="sse",
            sse_connect=f"/stream/{idx_asst}",
            sse_swap="message",
            sse_close="close",
            hx_swap="beforeend"
        ).__html__() +
        _chat_input().__html__()
    )


In [ ]:
#| export
#| eval: false

   

# ── helpers --------------------------------------------------------
def _sse(event: str, payload: str) -> str:
    """
    Return one correctly-formatted Server-Sent-Events block.

    Each logical message must be terminated with a *blank* line, otherwise
    the browser keeps buffering and the event never reaches the JS side.
    """
    # HTMX’ sse.js is happy with plain HTML, so we don’t wrap in JSON here.
    body = "\n".join(f"data: {line}" for line in payload.splitlines())
    return f"event: {event}\n{body}\n\n"


async def _stream_reply(idx: int) -> AsyncIterator[str] | None:  
        async with MSG_LOCK:
            if idx <= 0 or idx >= len(MSG):
                return 
        prompt_html = MSG[idx - 1]["content"]
    
        q: asyncio.Queue[str] = asyncio.Queue()
    
        # push() will be handed to ToolChatHook and to the agent itself
        def push(msg: str) -> None:  
            q.put_nowait(_sse("message", msg))
    
        async def _run() -> None:
            try:
                reply_html = await _assistant_html(prompt_html, push)
            except Exception as exc:
                reply_html = f"⚠ Internal error: {exc}"
            await q.put(_sse("message", reply_html))     # final HTML card
            await q.put("event: close\ndata:\n\n")       # tell HTMX to close
    
        asyncio.create_task(_run()) 
        
        # ⬅️  *this* is what StreamingResponse must consume
        async def streamer() -> AsyncIterator[str]:
            while True:                   # blocks until the queue gets data
                yield await q.get()

        return streamer()                 # ← DON’T forget this!     




In [ ]:
#| export
#| eval: false

@app_html.get("/stream/{idx}")
async def stream(idx: int):
    """
    Streaming endpoint used by the chat bubbles (`hx-ext="sse"`).
    HTMX opens the connection, waits for the first “message” event,
    swaps the payload into the bubble, then receives a “close” event
    and disposes the EventSource.
    """
    generator = await _stream_reply(idx)   # get the AsyncIterator ✅
    if generator is None:                  # invalid idx guard
        return HTMLResponse(
            status_code=status.HTTP_204_NO_CONTENT
        )
    return StreamingResponse(
       generator,
        media_type="text/event-stream",
    )

In [ ]:
#| export
#| eval: false




@app_html.post("/open-map")
async def open_map(
    # HTMX puts the value of `hx-vals` into the form as a *string*,
    # so accept it as str and deserialize ourselves.
    latlngs: str = ApiForm(...)
) -> HTMLResponse:
    """
    Render a small Leaflet preview.

    Parameters
    ----------
    latlngs : str (JSON)
        A JSON string that decodes to ``[[lat, lon], [lat, lon], …]``.

    Returns
    -------
    HTMLResponse
        A ready-to-open <dialog> card containing the map.
    """
    if isinstance(latlngs, str):
        coords = json.loads(latlngs)
    else:                       # HTMX/json-enc already gave us a list
        coords = latlngs
    assert all(isinstance(pt, list) and len(pt) == 2 for pt in coords)
    # -- 1️⃣ Validate & coerce ------------------------------------------------
    try:
        coords: List[List[float]] = json.loads(latlngs)
        assert (
            isinstance(coords, list)
            and all(isinstance(pt, list) and len(pt) == 2 for pt in coords)
        )
    except (json.JSONDecodeError, AssertionError, TypeError):
        # 400 instead of 500 makes the error obvious in the browser console
        raise HTTPException(status_code=400, detail="Bad latlngs payload")

    # -- 2️⃣ Build HTML -------------------------------------------------------
    coords_js = json.dumps(coords)  # safe to embed

    html = (
        Modal(
            ModalHeader("Route preview 🗺️"),
            ModalBody(
                "<div id='map' class='w-full h-80 rounded-xl'></div>",
                as_html=True,
            ),
            open=True,
        ).__html__()
                + f"""
        <link rel="stylesheet" href="{LEAFLET_CSS}"/>
        <script src="{LEAFLET_JS}"></script>
        <script>
          (function () {{
            const coords = {coords_js};
            const map = L.map('map', {{ zoomControl:false }}).fitBounds(coords);
            L.tileLayer('https://tile.openstreetmap.org/{{z}}/{{x}}/{{y}}.png',
                        {{ attribution:'© OpenStreetMap' }}).addTo(map);
            L.polyline(coords, {{ color:'#2563eb', weight:5 }}).addTo(map);
          }})();
        </script>
        """
    )

    return HTMLResponse(html, status_code=200, media_type="text/html")



In [ ]:
#| export
#| eval: false

@app_html.get("/parking-feed")
async def parking_feed():
    """
    Persistent SSE used by the camera loop.
    It never closes; every camera message is an already-formatted
    chat bubble HTML string produced by _sse("message", …).
    """
    async def streamer() -> AsyncIterator[str]:
        while True:
            yield await parking_q.get()

    return StreamingResponse(
        streamer(),
        media_type="text/event-stream",
    )
